Imports

In [5]:
# !pip install nest_asyncio
# import nest_asyncio
# nest_asyncio.apply()

In [1]:
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver import client, serializer

Sample Graph Creation

In [2]:
g = traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

In [6]:
# Insert a new vertex
new_vertex = g.addV('person').property('name', 'John').property('age', 30).next()

# Insert another vertex
another_vertex = g.addV('person').property('name', 'Alice').property('age', 25).next()

# Insert an edge between the vertices
g.V(new_vertex).addE('knows').to(g.V(another_vertex)).next()

e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': '3kn-3aw-4r9-6go'}}][4280-knows->8376]

Printing all nodes of the graph

In [13]:
g.V().valueMap(True).toList()

[{<T.id: 1>: 4208, <T.label: 4>: 'person', 'comName': ['value']},
 {<T.id: 1>: 4280, <T.label: 4>: 'person', 'name': ['John'], 'age': [30]},
 {<T.id: 1>: 8376, <T.label: 4>: 'person', 'name': ['Alice'], 'age': [25]}]

In [14]:
all_vertices = g.V().valueMap(True).toList()

# Print the nodes and their properties
for vertex_properties in all_vertices:
    print(vertex_properties)

{<T.id: 1>: 4208, <T.label: 4>: 'person', 'comName': ['value']}
{<T.id: 1>: 4280, <T.label: 4>: 'person', 'name': ['John'], 'age': [30]}
{<T.id: 1>: 8376, <T.label: 4>: 'person', 'name': ['Alice'], 'age': [25]}


Function to create a graph

In [24]:
def create_graph_node(label, properties):
    try:
        # Prepare the query to add a node
        query = g.addV(label)

        # Add properties to the node
        for key, value in properties.items():
            query = query.property(key, value)

        # Execute the query
        result_set = query.toList()
    finally:
        return result_set

In [25]:
node_label = "person"
node_properties = {
    "name": "John",
    "age": 30,
    "city": "New York",
}
created_node = create_graph_node(node_label, node_properties)
print("Created Node:", created_node)

Created Node: [v[16568]]


In [32]:
def get_node_properties_by_id(node_id):
    result = "Pending"
    try:
        # Prepare the query to get the node by its ID
        query = g.V(node_id)

        # Execute the query and get the properties of the node
        result_set = query.valueMap(True).toList()

        if result_set:
            # The query returns a list of dictionaries with properties as keys and lists of values.
            # Since we queried for a single node, we extract the first element from the list.
            node_properties = result_set[0]
            result =  node_properties
        else:
            result = None

    finally:
        return result

In [33]:
node_id = 16568

node_properties = get_node_properties_by_id(node_id)
if node_properties:
    print("Properties of Node with ID", node_id, ":", node_properties)
else:
    print("Node with ID", node_id, "not found.")

Properties of Node with ID 16568 : {<T.id: 1>: 16568, <T.label: 4>: 'person', 'name': ['John'], 'age': [30], 'city': ['New York']}


Create graph from CSV

In [35]:
import csv

file_path = "/home/smeet/Downloads/Bajaj Mall Data.csv"

# Open the CSV file in read mode
with open(file_path, mode='r', newline='') as csvfile:
    # Create a CSV reader object
    csv_reader = csv.reader(csvfile)
    
    # Read and process each row in the CSV file
    for row in csv_reader:
        # 'row' will be a list representing each row in the CSV file
        # You can access individual elements in the row using their indexes
        id = row[0]
        name = row[1]
        category = row[2]
        price = row[3]
        rating = row[4]
        
        # Now you can do whatever you want with the data in the row
        #print(f"ID: {id}, Name: {name}, Category: {category}, Price: {price}, Rating: {rating}")
        
        node_label = "product"
        node_properties = {
            "id": id,
            "name": name,
            "category": category,
            "price":price,
            "rating":rating
        }
        created_node = create_graph_node(node_label, node_properties)
        print("Created Node:", created_node)

Created Node: [v[4200]]
Created Node: [v[8304]]
Created Node: [v[4192]]
Created Node: [v[8288]]
Created Node: [v[12384]]
Created Node: [v[12400]]
Created Node: [v[20664]]
Created Node: [v[24760]]
Created Node: [v[28856]]
Created Node: [v[8256]]
Created Node: [v[16480]]
Created Node: [v[8296]]
Created Node: [v[12392]]
Created Node: [v[12352]]
Created Node: [v[16496]]
Created Node: [v[32952]]
Created Node: [v[16448]]
Created Node: [v[37048]]
Created Node: [v[16488]]
Created Node: [v[41144]]
Created Node: [v[20576]]
Created Node: [v[24672]]
Created Node: [v[28768]]
Created Node: [v[20544]]
Created Node: [v[20584]]
Created Node: [v[32864]]
Created Node: [v[24640]]
Created Node: [v[4264]]
Created Node: [v[36960]]
Created Node: [v[41056]]
Created Node: [v[8360]]


In [36]:
all_vertices = g.V().valueMap(True).toList()

# Print the nodes and their properties
for vertex_properties in all_vertices:
    print(vertex_properties)

{<T.id: 1>: 4160, <T.label: 4>: 'person', 'name': ['John'], 'age': [30], 'city': ['New York']}
{<T.id: 1>: 8256, <T.label: 4>: 'product', 'name': ['Haier 195 L 3 Star Direct Cool Single Door Refrigerator Black Brushline (HRD-1953CKS-E)'], 'price': ['13499'], 'rating': ['2.1'], 'id': ['8'], 'category': ['Fridge']}
{<T.id: 1>: 12352, <T.label: 4>: 'product', 'name': ['Samsung 198 3 Star Direct Cool Single Door Refrigerator (RR21A2D2YS8/HL)'], 'price': ['15799'], 'rating': ['2.2'], 'id': ['12'], 'category': ['Fridge']}
{<T.id: 1>: 16448, <T.label: 4>: 'product', 'name': ['OPPO F21 Pro 128 GB Storage Cosmic Black (8 GB RAM)'], 'price': ['20999'], 'rating': ['4.2'], 'id': ['15'], 'category': ['Smartphone']}
{<T.id: 1>: 20544, <T.label: 4>: 'product', 'name': ['Dell Intel Core i5 8 GB RAM/ 512 GB SSD/ Windows 11 Home/ 15.6 inch Laptop (Platinum Silver D560784WIN9S)'], 'price': ['55700'], 'rating': ['3.4'], 'id': ['22'], 'category': ['Laptop']}
{<T.id: 1>: 24640, <T.label: 4>: 'product', 'nam